In [ ]:


import os
import sys
from statistics import mean, median

STUDENTS_FILE = 'students.txt'

# --------------------- Helper functions for file handling ---------------------

def load_students():
    """Read students from STUDENTS_FILE and return a list of dicts.
    Each student dict: {'id': int, 'name': str, 'age': int, 'grade': str, 'marks': int}
    Malformed lines are skipped with a printed warning.
    """
    students = []
    if not os.path.exists(STUDENTS_FILE):
        return students

    with open(STUDENTS_FILE, 'r', encoding='utf-8') as f:
        for lineno, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            if len(parts) != 5:
                print(f"Warning: skipping malformed line {lineno} in {STUDENTS_FILE}: '{line}'")
                continue
            sid, name, age, grade, marks = parts
            # Try convert types with safe guards
            try:
                sid_i = int(sid)
                age_i = int(age)
                marks_i = int(marks)
                grade_s = grade.strip().upper()
                name_s = name.strip()
                students.append({'id': sid_i, 'name': name_s, 'age': age_i, 'grade': grade_s, 'marks': marks_i})
            except ValueError:
                print(f"Warning: skipping line with invalid data types at line {lineno}: '{line}'")
                continue
    return students


def save_students(students):
    """Overwrite STUDENTS_FILE with the provided list of student dicts."""
    with open(STUDENTS_FILE, 'w', encoding='utf-8') as f:
        for s in students:
            line = f"{s['id']},{s['name']},{s['age']},{s['grade']},{s['marks']}\n"
            f.write(line)


# --------------------- Validation helpers ---------------------

def input_int(prompt, min_val=None, max_val=None):
    """Prompt until a valid integer is entered and within optional bounds."""
    while True:
        val = input(prompt).strip()
        if not val:
            print("Input cannot be empty. Try again.")
            continue
        if not (val.lstrip('-').isdigit()):
            print("Please enter a valid integer.")
            continue
        num = int(val)
        if min_val is not None and num < min_val:
            print(f"Value must be >= {min_val}.")
            continue
        if max_val is not None and num > max_val:
            print(f"Value must be <= {max_val}.")
            continue
        return num


def input_non_numeric_string(prompt):
    """Prompt until non-empty and not purely numeric string is entered."""
    while True:
        s = input(prompt).strip()
        if not s:
            print("Input cannot be empty. Try again.")
            continue
        # if string is all numbers or contains only digits, reject
        if s.isdigit():
            print("Name cannot be purely numeric. Try again.")
            continue
        return s


def input_grade(prompt):
    """Prompt until a single uppercase letter A-F is entered."""
    while True:
        g = input(prompt).strip().upper()
        if len(g) != 1 or not g.isalpha():
            print("Grade must be a single letter (A-F).")
            continue
        if g < 'A' or g > 'F':
            print("Grade must be between A and F.")
            continue
        return g


# --------------------- CRUD operations ---------------------

def add_student():
    students = load_students()
    existing_ids = {s['id'] for s in students}

    print('\n--- Add Student ---')
    # ID must be integer and unique
    while True:
        sid = input_int('Enter ID (integer): ', min_val=1)
        if sid in existing_ids:
            print('ID already exists. Please enter a unique ID.')
            continue
        break

    name = input_non_numeric_string('Enter full name: ')
    age = input_int('Enter age: ', min_val=1)
    grade = input_grade('Enter grade (A-F): ')
    marks = input_int('Enter marks (0-100): ', min_val=0, max_val=100)

    students.append({'id': sid, 'name': name, 'age': age, 'grade': grade, 'marks': marks})
    save_students(students)
    print(f"Student {name} (ID {sid}) added successfully!\n")


def view_students():
    students = load_students()
    print('\n--- All Students ---')
    if not students:
        print('No records found.')
        return
    # Print header
    print(f"{'ID':<6}{'Name':<25}{'Age':<6}{'Grade':<8}{'Marks':<8}")
    print('-' * 55)
    for s in students:
        print(f"{s['id']:<6}{s['name']:<25}{s['age']:<6}{s['grade']:<8}{s['marks']:<8}")
    print(f"\nTotal records: {len(students)}\n")


def search_student():
    students = load_students()
    if not students:
        print('\nNo records to search.')
        return
    print('\n--- Search Student ---')
    choice = input('Search by (1) ID or (2) Name (partial): ').strip()
    if choice == '1':
        sid = input_int('Enter ID to search: ', min_val=1)
        found = [s for s in students if s['id'] == sid]
    else:
        key = input('Enter name or part of name to search (case-insensitive): ').strip().lower()
        found = [s for s in students if key in s['name'].lower()]

    if not found:
        print('No matching records found.\n')
        return

    print(f"Found {len(found)} record(s):")
    for s in found:
        print(f"ID: {s['id']} | Name: {s['name']} | Age: {s['age']} | Grade: {s['grade']} | Marks: {s['marks']}")
    print()


def update_student():
    students = load_students()
    if not students:
        print('\nNo records to update.')
        return
    print('\n--- Update Student ---')
    sid = input_int('Enter ID of student to update: ', min_val=1)
    idx = next((i for i, s in enumerate(students) if s['id'] == sid), None)
    if idx is None:
        print('Student with that ID not found.\n')
        return
    s = students[idx]
    print(f"Current data: ID {s['id']}, Name: {s['name']}, Age: {s['age']}, Grade: {s['grade']}, Marks: {s['marks']}")
    print('Enter new values or press Enter to keep current.')

    # selective updates
    new_name = input('New name: ').strip()
    if new_name:
        if new_name.isdigit():
            print('Invalid name; keeping old name.')
        else:
            s['name'] = new_name

    new_age = input('New age: ').strip()
    if new_age:
        if new_age.lstrip('-').isdigit():
            age_i = int(new_age)
            if age_i > 0:
                s['age'] = age_i
            else:
                print('Invalid age; keeping old age.')
        else:
            print('Invalid age input; keeping old age.')

    new_grade = input('New grade (A-F): ').strip()
    if new_grade:
        ng = new_grade.upper()
        if len(ng) == 1 and ng.isalpha() and 'A' <= ng <= 'F':
            s['grade'] = ng
        else:
            print('Invalid grade; keeping old grade.')

    new_marks = input('New marks (0-100): ').strip()
    if new_marks:
        if new_marks.isdigit():
            m_i = int(new_marks)
            if 0 <= m_i <= 100:
                s['marks'] = m_i
            else:
                print('Marks out of range; keeping old marks.')
        else:
            print('Invalid marks input; keeping old marks.')

    # write back
    students[idx] = s
    save_students(students)
    print('Student record updated successfully.\n')


def delete_student():
    students = load_students()
    if not students:
        print('\nNo records to delete.')
        return
    print('\n--- Delete Student ---')
    sid = input_int('Enter ID of student to delete: ', min_val=1)
    idx = next((i for i, s in enumerate(students) if s['id'] == sid), None)
    if idx is None:
        print('Student with that ID not found.\n')
        return
    s = students[idx]
    confirm = input(f"Are you sure you want to delete {s['name']} (ID {s['id']})? (y/n): ").strip().lower()
    if confirm == 'y':
        students.pop(idx)
        save_students(students)
        print('Student deleted successfully.\n')
    else:
        print('Deletion cancelled.\n')


# --------------------- Analysis functions ---------------------

def analyze_data():
    students = load_students()
    print('\n--- Data Analysis ---')
    if not students:
        print('No data available for analysis.\n')
        return

    marks = [s['marks'] for s in students if isinstance(s.get('marks'), int)]
    if not marks:
        print('No valid marks present in data.\n')
        return

    # Basic insights
    avg_marks = mean(marks)
    med_marks = median(marks)
    highest = max(marks)
    lowest = min(marks)
    top_performers = [s for s in students if s['marks'] == highest]
    below_avg_count = sum(1 for m in marks if m < avg_marks)

    print(f"Average Marks: {avg_marks:.2f}")
    print(f"Median Marks: {med_marks}")
    tp = ', '.join(f"{p['name']} ({p['marks']})" for p in top_performers)
    print(f"Top Performer(s): {tp}")
    print(f"Students Below Average: {below_avg_count}")
    print(f"Highest Marks: {highest} | Lowest Marks: {lowest}")

    # Custom analytical insights
    grade_distribution(students)
    pass_rate(students)
    percentile_insight(students)

    print()


def grade_distribution(students):
    """Custom Insight 1: shows how many students per grade and percentage."""
    total = len(students)
    dist = {}
    for s in students:
        g = s.get('grade', 'Unknown')
        dist[g] = dist.get(g, 0) + 1
    print('\nGrade Distribution:')
    for g in sorted(dist.keys()):
        count = dist[g]
        pct = (count / total) * 100
        print(f" Grade {g}: {count} student(s) ({pct:.1f}%)")


def pass_rate(students, passing_mark=40):
    """Custom Insight 2: compute pass rate and list students failing."""
    total = len(students)
    passed = [s for s in students if isinstance(s.get('marks'), int) and s['marks'] >= passing_mark]
    failed = [s for s in students if isinstance(s.get('marks'), int) and s['marks'] < passing_mark]
    rate = (len(passed) / total) * 100
    print(f"\nPass Rate (>= {passing_mark}): {rate:.1f}% ({len(passed)}/{total})")
    if failed:
        print(' Students failing: ' + ', '.join(f"{f['name']} ({f['marks']})" for f in failed))


def percentile_insight(students):
    """Custom Insight 3: show how many students are in top 10% and bottom 10% (by marks).
    If there are few students, insights are adjusted sensibly.
    """
    marks_sorted = sorted([s['marks'] for s in students])
    n = len(marks_sorted)
    if n < 3:
        print('\nPercentile Insight: Not enough data for percentile-based insights.')
        return
    # compute thresholds
    top_thresh_index = max(int(n * 0.9) - 1, 0)
    bottom_thresh_index = min(int(n * 0.1), n-1)
    top_thresh = marks_sorted[top_thresh_index]
    bottom_thresh = marks_sorted[bottom_thresh_index]
    top_students = [s for s in students if s['marks'] >= top_thresh]
    bottom_students = [s for s in students if s['marks'] <= bottom_thresh]
    print(f"\nTop ~10% threshold: {top_thresh}; Students in top ~10%: {len(top_students)}")
    print(f"Bottom ~10% threshold: {bottom_thresh}; Students in bottom ~10%: {len(bottom_students)}")


# --------------------- Main program flow ---------------------

def print_menu():
    print('''\n====== Smart Student Record Analyzer ======\n'
1. Add Student\n2. View All Students\n3. Search Student\n4. Update Student\n5. Delete Student\n6. Analyze Data\n7. Exit\n''')


def main():
    while True:
        print_menu()
        choice = input('Enter your choice: ').strip()
        if choice == '1':
            add_student()
        elif choice == '2':
            view_students()
        elif choice == '3':
            search_student()
        elif choice == '4':
            update_student()
        elif choice == '5':
            delete_student()
        elif choice == '6':
            analyze_data()
        elif choice == '7':
            print('\nThank you for using Smart Student Record Analyzer. Goodbye!')
            break
        else:
            print('Invalid choice. Please enter a number from 1 to 7.')


if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print('\n\nInterrupted. Exiting gracefully. Goodbye!')
        sys.exit(0)



====== Smart Student Record Analyzer ======
'
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Exit



Enter your choice:  1



--- Add Student ---


Enter ID (integer):  450826
Enter full name:  Madan Lal
Enter age:  30
Enter grade (A-F):  A
Enter marks (0-100):  92


Student Madan Lal (ID 450826) added successfully!


====== Smart Student Record Analyzer ======
'
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Exit



...


--- Search Student ---


Search by (1) ID or (2) Name (partial):  450826
Enter name or part of name to search (case-insensitive):  Madan Lal


Found 1 record(s):
ID: 450826 | Name: Madan Lal | Age: 30 | Grade: A | Marks: 92


====== Smart Student Record Analyzer ======
'
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Exit



Enter your choice:  6



--- Data Analysis ---
Average Marks: 74.50
Median Marks: 76.0
Top Performer(s): Madan Lal (92)
Students Below Average: 3
Highest Marks: 92 | Lowest Marks: 55

Grade Distribution:
 Grade A: 2 student(s) (33.3%)
 Grade B: 1 student(s) (16.7%)
 Grade C: 1 student(s) (16.7%)
 Grade D: 1 student(s) (16.7%)
 Grade E: 1 student(s) (16.7%)

Pass Rate (>= 40): 100.0% (6/6)

Top ~10% threshold: 81; Students in top ~10%: 2
Bottom ~10% threshold: 55; Students in bottom ~10%: 1


====== Smart Student Record Analyzer ======
'
1. Add Student
2. View All Students
3. Search Student
4. Update Student
5. Delete Student
6. Analyze Data
7. Exit

